In [ ]:
import pymysql
import os
import pymysql
from pymysql.cursors import DictCursor, SSCursor  # DictCursor = nice columns; SSCursor = stream big results
import pandas as pd

In [ ]:
# Connection parameters
host = 'localhost'
port = 3307  # Replace with the correct port number
user = 'hhims'
password = 'JudbGaQLG!7bpJG6'
database = 'hhimsv3'

# Establish a connection
connection = pymysql.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database,
    charset="utf8mb4",  
    cursorclass=DictCursor,
    autocommit=True, 
    connect_timeout=10
)

In [ ]:

# Tip: utf8mb4 + timezone handling (if needed)
conn = pymysql.connect(
    host=host, port=port, user=user, password=password, database=database,
    charset="utf8mb4", cursorclass=DictCursor, autocommit=True, connect_timeout=10
)

try:
    with conn.cursor() as cur:
        # Avoid SELECT * in analytics code; name the columns you actually need
        cur.execute("""
            SELECT OPDID,Complaint,Doctor,YEAR (DateTimeOfVisit) AS visit_year ,DATE (DateTimeOfVisit) AS visit_date
            FROM opd_visits
            
        """)
        rows = cur.fetchall()
        #print("Top 5 rows:")
        #for r in rows:
        #    print(r)

    # Direct to pandas with proper column names (thanks to DictCursor)
    df = pd.DataFrame(rows)
    
    # Or let pandas pull straight from the connection:
    # df = pd.read_sql("SELECT OPDID, PID, DateTimeOfVisit FROM opd_visits LIMIT 5", conn)

finally:
    conn.close()


In [ ]:
df.head()

In [112]:
# write df to csv
#df.to_csv("opd_visits_data.csv", index=False)
df = pd.read_csv("opd_visits_data.csv")

In [128]:
df.head(1)

,OPDID,Complaint,Doctor,visit_year,visit_date,refined_rfe
0,1,"Blood test,",2,2021,2021-08-19,"Blood test,"


In [113]:
len(df)

935667

In [ ]:
Cough = ["Cough,","Cough","c","Fever,Cough,","cold","cough 3d","COUGH 2D","cough,Fever,","Cough,acute nasopharyngitis (common cold),",
         "cough 1d","Cough 1w","F,C","cough 4d","cough,cold","Sneezing/nasal congestion,","fever , cough",
         "cough 5d","cough and cold","cough,sob","ough","caugh cold","c,c","cough 3 days","cough 1 week",
         "cough sob"]

In [131]:
Fever = ["Fever,","Fever,Cough,","fever","fever 1D","f1d","F","fever 3d","f 1D","fever1d",
         "fever 2d","f3d","fever d 1,cough","fever 4d","fever3d","Fever,3d","fever4D","Fever,acute nasopharyngitis (common cold),",
         "fever d 1","fever2d","fever d 3,cough","F,C","Fever,1d","fever 3 days","fever , cough",
         "fever 1 day","fever,Vomiting,","fever 5D","fever,cough 1d","fev1d","fever 1 d",
         "fever,cough","fever 4 days","Fever,3d,Cough,","F 2d","Fever,1d,Cough,","Fever,4d","fever 3 d",
         "Fever,3d","fever4D","Fever,acute nasopharyngitis (common cold),","fever d 1","fever2d",
         "fever d 3,cough","F,C","Fever,1d","fever 3 days","fever d 3","fever , cough","fever 1 day",
         "fever,Vomiting,","fever 5D","fever,cough 1d","fev1d","fever 1 d","fever,cough",
         "fever 4 days","Fever,3d,Cough,","F 2d","Fever,1d,Cough,","Fever,4d","fever 3 d",
         "fever cough",]

In [147]:
Trauma = ["Trauma/injury NOS,","trauma",
        "Blunt trauma,","TRAUMA TO HEAD"]

In [148]:
Fall = ["Fall on ground,","H/O fall","Fall from height,","HO FALL","H/O  FALL",]

In [ ]:
# Rash and dermat conditions

In [ ]:
Rash = ["Rash localized,","rash","skin","Rash generalized,","skin rash"]

In [ ]:
#Animal_trauma = ["dog bite","Cat scratch","Animal bite,","cat bite","rat bite"]

In [150]:
abdominal_pain = ["abd pain","ab pain",
      "abdominal pain","abd.pain","abd. pain"]

In [136]:
Wheezing = ["wheezing","Wheezing,"]

In [ ]:
#Eye_symptoms = ["eye","poor vision","Red eye,","squint","Eye pain,"]

In [118]:
df.head(1)

,OPDID,Complaint,Doctor,visit_year,visit_date
0,1,"Blood test,",2,2021,2021-08-19


In [161]:
# clean steps simples comma
df['Complaint'] = df['Complaint'].str.lower()
df['refined_rfe'] = df['refined_rfe'].str.rstrip(',')  



In [162]:
# refine for all above complaints
df['refined_rfe'] = df['Complaint']  # start by copying original column

df.loc[df['Complaint'].str.contains('|'.join(Cough), case=False, na=False), 'refined_rfe'] = "cough"
df.loc[df['Complaint'].str.contains('|'.join(Fever), case=False, na=False), 'refined_rfe'] = "fever"
df.loc[df['Complaint'].str.contains('|'.join(Fall), case=False, na=False), 'refined_rfe'] = "fall"
df.loc[df['Complaint'].str.contains('|'.join(Rash), case=False, na=False), 'refined_rfe'] = "rash"
#df.loc[df['Complaint'].str.contains('|'.join(Animal_trauma), case=False, na=False), 'refined_rfe'] = "animal trauma"
df.loc[df['Complaint'].str.contains('|'.join(abdominal_pain), case=False, na=False), 'refined_rfe'] = "abdominal pain"
df.loc[df['Complaint'].str.contains('|'.join(Wheezing), case=False, na=False), 'refined_rfe'] = "wheezing"
#df.loc[df['Complaint'].str.contains('|'.join(Eye_symptoms), case=False, na=False), 'refined_rfe'] = "eye symptoms"
    

C:\Users\Chinthaka\AppData\Local\Temp\ipykernel_18156\488917506.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['Complaint'].str.contains('|'.join(Cough), case=False, na=False), 'refined_rfe'] = "cough"
C:\Users\Chinthaka\AppData\Local\Temp\ipykernel_18156\488917506.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['Complaint'].str.contains('|'.join(Fever), case=False, na=False), 'refined_rfe'] = "fever"


In [163]:
df.head()

,OPDID,Complaint,Doctor,visit_year,visit_date,refined_rfe
0,1,"blood test,",2,2021,2021-08-19,"blood test,"
1,2,"other laboratory test nec,",2,2021,2021-08-19,cough
2,3,test,2,2021,2021-08-24,test
3,4,test,2,2021,2021-09-18,test
4,5,test,4,2021,2021-11-23,test


In [164]:
# total unique complaints
unique_complaints_count = df['Complaint'].nunique()
unique_complaints_count

130255

In [173]:
# unique complaints
unique_complaints_df = df['refined_rfe'].value_counts()

unique_complaints_df.to_csv("unique_complaints_refined.csv", index=True)
